In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

### FewShotPromptTemplate

- 답변의 예시를 보여줘서 **AI 가 답변의 예시를 따라서 답변**할 수 있도록 하는 방법입니다.
- `OneShot` 은 예시가 하나일때 사용입니다.
- `FewShot` 은 과거의 이력을 모아서 여러개의 예시를 사용하는 것입니다.(사람이 하는 논리적인 추론과정을 먼저 보여주고 AI가 답변하게끔 유도하는 방법입니다.)
- `ZeroShot` 은 아무런 예시를 보여주지 않는 형태를 말합니다.

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

question = "비트코인의 단위는 무엇인가요?"

print(llm.invoke(question).content)

비트코인의 기본 단위는 '비트코인'이며, 기호로는 BTC라고 표기합니다. 비트코인은 더 작은 단위로도 나눌 수 있는데, 주요 단위로는 '밀리비트코인'(mBTC), '마이크로비트코인'(μBTC, 또는 비트), 그리고 '사토시'가 있습니다. 사토시는 비트코인의 가장 작은 단위로, 1비트코인은 1억 사토시(0.00000001 BTC)와 같습니다. 이러한 단위들은 비트코인의 거래 시 소액 거래를 용이하게 하기 위해 사용됩니다.


In [2]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

examples = [
    {
        "question": "비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    """,
    },
    {
        "question": "리플과 카르다노의 창립자가 같은 대륙 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 리플의 창립자는 누구인가요?
    중간 답변: 리플의 창립자는 크리스 라센입니다.
    추가 질문: 크리스 라센은 어느 대륙 출신인가요?
    중간 답변: 크리스 라센은 북미 출신입니다.
    추가 질문: 카르다노의 창립자는 누구인가요?
    중간 답변: 카르다노의 창립자는 찰스 호스킨슨입니다.
    추가 질문: 찰스 호스킨슨은 어느 대륙 출신인가요?
    중간 답변: 찰스 호스킨슨은 북미 출신입니다.
    최종 답변은: 예
    """,
    },
    {
        "question": "폴리곤과 솔라나 중 거래 속도가 더 빠른 네트워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 폴리곤 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 폴리곤 네트워크는 초당 약 7,000 거래를 처리할 수 있습니다.
    추가 질문: 솔라나 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 솔라나 네트워크는 초당 약 65,000 거래를 처리할 수 있습니다.
    최종 답변은: 솔라나
    """,
    },
    {
        "question": "체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    """,
    },
]

#### OneShot

In [6]:
example_prompt = PromptTemplate.from_template(
    "Question:\n {question}\nAnswer:\n {answer}"
)

# 하나만 찾아서 예시를 주었기 때문에 OneShot 입니다.
print(
    example_prompt.format(
        **examples[0]  # ** 문법은 언패킹 문법입니다. {} 를 제거해줍니다.
    )
)

Question:
 비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    


#### FewShow

In [17]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n {question}\nAnswer:\n ",
    input_variables=["question"],
)

question = "니어프로토콜은 어떤 체인인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
 비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    

Question:
 리플과 카르다노의 창립자가 같은 대륙 출신인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 리플의 창립자는 누구인가요?
    중간 답변: 리플의 창립자는 크리스 라센입니다.
    추가 질문: 크리스 라센은 어느 대륙 출신인가요?
    중간 답변: 크리스 라센은 북미 출신입니다.
    추가 질문: 카르다노의 창립자는 누구인가요?
    중간 답변: 카르다노의 창립자는 찰스 호스킨슨입니다.
    추가 질문: 찰스 호스킨슨은 어느 대륙 출신인가요?
    중간 답변: 찰스 호스킨슨은 북미 출신입니다.
    최종 답변은: 예
    

Question:
 폴리곤과 솔라나 중 거래 속도가 더 빠른 네트워크는 무엇인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 폴리곤 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 폴리곤 네트워크는 초당 약 7,000 거래를 처리할 수 있습니다.
    추가 질문: 솔라나 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 솔라나 네트워크는 초당 약 65,000 거래를 처리할 수 있습니다.
    최종 답변은: 솔라나
    

Question:
 체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답

In [18]:
chain = prompt | llm

print(chain.invoke(question).content)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: 니어프로토콜은 어떤 기능을 가지고 있나요?  
중간 답변: 니어프로토콜은 분산형 애플리케이션(dApp)을 구축할 수 있는 블록체인 플랫폼입니다.  
추가 질문: 니어프로토콜의 합의 알고리즘은 무엇인가요?  
중간 답변: 니어프로토콜은 나이트샤드(Nightshade)라는 샤딩 기술을 사용하는 합의 알고리즘을 채택하고 있습니다.  
최종 답변은: 니어프로토콜은 분산형 애플리케이션을 구축할 수 있는 블록체인 플랫폼입니다.


### Example Selector

- 예제가 많은 경우 프롬프트에 포함된 예제를 선택할 수 있습니다. 질문과 유사도를 판단해서 전체 예제를 프롬프트에 포함시키지않고 **선택해서** 포함합니다.
- SemanticSimilarityExampleSelector : 질문과 유사도 계산을 해서 유사도가 높은 몇 개를 선택해줍니다.

프롬프트의 비용을 줄일 수 있는 방법 중에 하나입니다.

In [22]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
Chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 예시 목록입니다.
    examples,
    # 유사성을 측정하는 임베딩 클래스입니다.
    OpenAIEmbeddings(),
    # 벡터 데이터베이스 클레스 입니다.
    Chroma,
    # 선택할 예시의 갯수입니다.
    k=1,
)

question = "니어프로토콜은 어떤 체인인가요?"
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시입니다 : \n{question}\n")
for example in selected_examples:
    print(f"question:\n{example['question']}")
    print(f"answer:\n{example['answer']}")

입력에 가장 유사한 예시입니다 : 
니어프로토콜은 어떤 체인인가요?

question:
체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    


In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

examples = [
    {
        "question": "비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    """,
    },
    {
        "question": "리플과 카르다노의 창립자가 같은 대륙 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 리플의 창립자는 누구인가요?
    중간 답변: 리플의 창립자는 크리스 라센입니다.
    추가 질문: 크리스 라센은 어느 대륙 출신인가요?
    중간 답변: 크리스 라센은 북미 출신입니다.
    추가 질문: 카르다노의 창립자는 누구인가요?
    중간 답변: 카르다노의 창립자는 찰스 호스킨슨입니다.
    추가 질문: 찰스 호스킨슨은 어느 대륙 출신인가요?
    중간 답변: 찰스 호스킨슨은 북미 출신입니다.
    최종 답변은: 예
    """,
    },
    {
        "question": "폴리곤과 솔라나 중 거래 속도가 더 빠른 네트워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 폴리곤 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 폴리곤 네트워크는 초당 약 7,000 거래를 처리할 수 있습니다.
    추가 질문: 솔라나 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 솔라나 네트워크는 초당 약 65,000 거래를 처리할 수 있습니다.
    최종 답변은: 솔라나
    """,
    },
    {
        "question": "체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    """,
    },
]

embeddings = OpenAIEmbeddings()
chroma_db = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=chroma_db,
    k=2,
)

example_prompt = PromptTemplate.from_template(
    "Question:\n {question}\nAnswer:\n {answer}"
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n {question}\nAnswer:\n ",
    input_variables=["question"],
)

question = "니어프로토콜은 어떤 체인인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

Question:
 체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    

Question:
 체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    

Question:
 니어프로토콜은 어떤 체인인가요?
Answer:
 


In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)


prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n {question}\nAnswer:\n ",
    input_variables=["question"],
)

chain = prompt | llm | StrOutputParser()

answer = chain.stream(
    {
        "question": "니어프로토콜과 아카시네트워크는 어떤 체인인가요?"
    }  # 질문이 올바르지 않으면 example_selector 처럼 원하는 형식의 답변을 받지 못 할 수도 있습니다.
)

for token in answer:
    print(token, end="", flush=True)

이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 니어프로토콜은 어떤 플랫폼인가요?
    중간 답변: 니어프로토콜은 확장성이 뛰어난 블록체인 플랫폼으로, 스마트 계약과 분산 애플리케이션(dApps)을 지원합니다.
    추가 질문: 아카시네트워크는 어떤 플랫폼인가요?
    중간 답변: 아카시네트워크는 분산형 클라우드 컴퓨팅 플랫폼으로, 블록체인 기술을 사용하여 서버리스 컴퓨팅 환경을 제공합니다.
    최종 답변은: 니어프로토콜과 아카시네트워크는 각각 다른 목적을 가진 체인입니다. 니어프로토콜은 스마트 계약과 dApps 지원을 위한 블록체인 플랫픔이며, 아카시네트워크는 분산형 클라우드 컴퓨팅을 제공하는 플랫폼입니다.